In [114]:
# Import Python Libraries
from time import sleep
import json
import os
import smtplib
import socket
import sys
from collections import namedtuple
from dataclasses import dataclass
from email.mime.text import MIMEText
from typing import Any, Dict, Tuple

# import Qumulo REST libraries:
from qumulo.rest_client import RestClient
from qumulo.lib.request import RequestError

In [33]:
#  _   _ _____ _     ____  _____ ____  ____
# | | | | ____| |   |  _ \| ____|  _ \/ ___|
# | |_| |  _| | |   | |_) |  _| | |_) \___ \
# |  _  | |___| |___|  __/| |___|  _ < ___) |
# |_| |_|_____|_____|_|   |_____|_| \_\____/

def load_json(file: str):
    """
    Load a file and ensure that it's valid JSON.
    """
    try:
        file_fh = open(file, 'r')
        data = json.load(file_fh)
        return data
    except ValueError as error:
        sys.exit(f'Invalid JSON file: {file}. Error: {error}')
    finally:
        file_fh.close()

def load_config(config_file: str):
    """
    Load json as json dictionary-like object for parsing.
    """    
    if os.path.exists(config_file):
        return load_json(config_file)
    else:
        sys.exit(f'Configuration file "{config_file}" does not exist.')

In [34]:
#   ___                           _    ____ ___ 
#  / _ \ _   _  ___ _ __ _   _   / \  |  _ \_ _|
# | | | | | | |/ _ \ '__| | | | / _ \ | |_) | | 
# | |_| | |_| |  __/ |  | |_| |/ ___ \|  __/| | 
#  \__\_\\__,_|\___|_|   \__, /_/   \_\_|  |___|
#                        |___/                  

def cluster_login(api_hostname, api_username, api_password):
    """
    Accept api_hostname, api_username and api_password as parameters. Log into
    cluster via Qumulo Rest API. Return rest_client for all future API calls.
    """
    rest_client = RestClient(api_hostname, 8000)
    rest_client.login(api_username, api_password)

    return rest_client

def get_cluster_name(rest_client):
    """
    Query API for cluster name. Return cluster name as string.
    """
    cluster_name = rest_client.cluster.get_cluster_conf()['cluster_name']

    return cluster_name

def get_qq_version(rest_client):
    """
    Query API for Qumulo Core version. Return version as string.
    """

    qq_version = rest_client.version.version()['revision_id']

    return qq_version

def get_cluster_time(rest_client):
    """
    Get current cluster time and return as cluster_time.
    """
    
    cluster_time = rest_client.time_config.get_time_status()['time']
    
    return cluster_time

def get_cluser_uuid(rest_client):
    """
    Query API for cluster UUID number. Return UUID as string.
    """
    cluster_uuid = rest_client.node_state.get_node_state()['cluster_id']
    
    return cluster_uuid

def retrieve_status_of_cluster_nodes(rest_client):
    """
    Accept rest_client object to query via API call to retrieve info/status for
    nodes. Parse through information and record relevant information. Return
    dict object to later dump as json.
    """
    node_relevant_fields = [
        'id',
        'node_status',
        'node_name',
        'uuid',
        'model_number',
        'serial_number',
    ]

    temp_list = []
    for num in range(len(rest_client.cluster.list_nodes())):
        new_dict = {}
        for k,v in rest_client.cluster.list_nodes()[num].items():
            if k in node_relevant_fields:
                new_dict[k] = v
        temp_list.append(new_dict)
    
    status_of_nodes = {}
    status_of_nodes['nodes'] = temp_list

    return status_of_nodes

def retrieve_status_of_cluster_drives(rest_client):
    """
    Accept rest_client object to query via API call to retrieve info/status for
    drives. Parse through information and record relevant information. Return
    dict object to later dump as json.
    """
    drive_relevant_fields = [
        'id',
        'node_id',
        'slot',
        'state',
        'slot_type',
        'disk_type',
        'disk_model',
        'disk_serial_number',
        'capacity',
    ]

    temp_list = []
    for num in range(len(rest_client.cluster.get_cluster_slots_status())):
        new_dict = {}
        for k,v in rest_client.cluster.get_cluster_slots_status()[num].items():
            if k in drive_relevant_fields:
                new_dict[k] = v
        temp_list.append(new_dict)

    status_of_drives = {}
    status_of_drives['drives'] = temp_list

    return status_of_drives

def combine_statuses_formatting(status_of_nodes, status_of_drives):
    """
    In order to adhere to proper json formatting, this func will combine the
    two status_of_nodes and status_of_drives dictionary objects into one
    single dictionary object and return this as cluster_status.
    """
    status_of_nodes['drives'] = status_of_drives['drives']    
    cluster_status = status_of_nodes
    
    return cluster_status

In [35]:
# rest_client = cluster_login(API_HOSTNAME, API_USERNAME, API_PASSWORD)

In [36]:
# rest_client.ad.list_ad()

In [37]:
# rest_client.cluster.list_nodes()

In [38]:
# rest_client.cluster.list_nodes()[0]['node_status']

In [39]:
# rest_client.node_state.get_node_state()

In [40]:
# rest_client.cluster.get_cluster_slots_status()

In [41]:
# if 'cluster_state.json' in os.listdir():
#     os.rename('cluster_state.json','cluster_state_previous.json')
#     PREVIOUS_EXISTED = True

# with open('cluster_state.json', 'w') as f:
#     for data in rest_client.cluster.list_nodes():
#         f.writelines(data)

In [42]:
# for item in rest_client.cluster.list_nodes():
#     print(item)

In [43]:
# rest_client.cluster.list_nodes()[:2]

In [44]:
# rest_client.cluster.list_nodes()[0]

In [45]:
# node_relevant_fields = ['id','node_status','node_name','uuid','model_number','serial_number']
# drive_relevant_fields = ['id','node_id','slot','state','slot_type','disk_type','disk_model','disk_serial_number','capacity']

In [46]:
# newlist = []
# for entry in range(len(rest_client.cluster.list_nodes())):
#     newdict = {}
#     for k,v in rest_client.cluster.list_nodes()[entry].items():
#         if k in node_relevant_fields:
#             newdict[k] = v
#     newlist.append(newdict)

In [47]:
# for entry in range(len(rest_client.cluster.list_nodes())):
#     print(entry)

In [48]:
# rest_client.cluster.list_nodes()[1].items()

In [49]:
# status_of_nodes = retrieve_status_of_cluster_nodes(rest_client)
# status_of_nodes

In [50]:
# status_of_drives = retrieve_status_of_cluster_drives(rest_client)
# status_of_drives

In [51]:
#   ____                                     ____  _        _            
#  / ___|___  _ __ ___  _ __   __ _ _ __ ___/ ___|| |_ __ _| |_ ___  ___ 
# | |   / _ \| '_ ` _ \| '_ \ / _` | '__/ _ \___ \| __/ _` | __/ _ \/ __|
# | |__| (_) | | | | | | |_) | (_| | | |  __/___) | || (_| | ||  __/\__ \
#  \____\___/|_| |_| |_| .__/ \__,_|_|  \___|____/ \__\__,_|\__\___||___/
#                      |_|                                               

def check_for_previous_state(cluster_status):
    """
    If cluster_state.json exists, rename it to cluster_state_previous.json.
    Regardless of this, also create cluster_state.json and write node + drive
    statuses to file. Return boolean for previous_existed.
    """
    if 'cluster_state.json' in os.listdir():
        os.rename('cluster_state.json','cluster_state_previous.json')
        previous_existed = True
    else:
        previous_existed = False

    with open('cluster_state.json', 'w') as f:
        json.dump(cluster_status, f, indent=4)

    return previous_existed

def compare_states():
    """
    Only being ran if previous_existed is true, this func will compare the
    json files for the previous and current cluster state. Return bool for
    whether or not the data has changed. Return bool for if changes were found.
    """    

    file1 = 'cluster_state.json'
    file2 = 'cluster_state_previous.json'

    with open(file1) as f1, open(file2) as f2:
        data1, data2 = json.load(f1), json.load(f2)
        changes = data1 == data2

    # XXX: can potentially remove all of this
    if changes:
        print('Changes found!! Scanning for unhealthy objects.') # XXX: Later remove
    else:
        print('Changes not found! Not scanning for unhealthy objects') # XXX: Later remove

    return changes

def check_for_unhealthy_objects():
    """ 
    Scan the cluster_state.json file to determine whether or not there are 
    unhealthy objects. If there are unhealthy objects, append the data to
    new dict object called alert_data, which will later be used to populate
    the alert. Also return whether or not cluster is healthy as bool. 
    """
    healthy = True

    with open('cluster_state_TEST.json') as f:  # XXX: Later change value to 'cluster_state.json'
        data = json.load(f)
        alert_data = {}
        counter = 1
        
        # scan through json for offline nodes
        for dictobj in data['nodes']:
            for k,v in dictobj.items():
                if k == 'node_status':
                    if v != 'online':
                        print('ALERT!! UNHEALTHY NODE FOUND.') # XXX: Later remove
                        alert_data[f'Event {counter}'] = dictobj
                        counter += 1
                        healthy = False
        # scan through json for unhealthy drives
        for dictobj in data['drives']:
            for k,v in dictobj.items():
                if k == 'state':
                    if v != 'healthy':
                        print('ALERT!! UNHEALTHY DRIVE FOUND.') # XXX: Later remove 
                        alert_data[f'Event {counter}'] = dictobj
                        counter += 1
                        healthy = False
            
    if healthy:
        print('No unhealthy changes found.')

    # print(f'alert data: {alert_data}') # XXX: later remove

    return alert_data, healthy

In [52]:
# alert_data, healthy = check_for_unhealthy_objects()

In [53]:
# json1 = json.dumps('cluster_state.json', sort_keys=True)
# json2 = json.dumps('cluster_state_TEST.json', sort_keys=True)
# print(f'comparison: {json1 != json2}')

In [54]:
# with open('cluster_state.json') as f1, open('cluster_state_TEST.json') as f2:
#     data1 = json.load(f1)
#     data2 = json.load(f2)
#     something = data1 == data2
#     print(something)

In [55]:
# node_event = 'A node has gone offline.'

# for item in alert_data:
#     for k,v in alert_data[item].items():
#         if k == 'node_status':
#             print(node_event)

In [56]:
# rest_client.cluster.list_node(2)['uuid']

In [57]:
# for val in alert_data['Event 2'].items():
#     print(val)

In [58]:
# qq_version = get_qq_version(rest_client)
# cluster_name = get_cluster_name(rest_client)
# cluser_uuid = get_cluser_uuid(rest_client)
# cluster_time = get_cluster_time(rest_client)
# alert_header = '=' * 18 + ' CLUSTER EVENT ALERT! ' + '=' * 18
# email_alert = f"""{alert_header}\nUnhealthy object(s) found. See below for info
# and engage Qumulo Support in your preferred fashion.

# Cluster name: {cluster_name}
# Cluster UUID: {cluster_uuid}
# Approx. time: {cluster_time}

# Event(s) found:
# """

# node_event_heading = '=' * 15 + ' A node has gone offline. ' + '=' * 15
# drive_event_heading = '=' * 15 + ' A drive is no longer healthy. ' + '=' * 15

# for item in alert_data:
#     for k,v in alert_data[item].items():
#         if k == 'node_status':    # this is a node alert
#             email_alert += node_event_heading
#             node_alert_text = f"""
#             Node number: {alert_data[item]['id']}
#             Node status: {alert_data[item]['node_status']}
#             Serial Number: {alert_data[item]['serial_number']}
#             Node UUID: {alert_data[item]['uuid']}           
#             Node Type: {alert_data[item]['model_number']}
#             Qumulo Core Version: {qq_version}
#             """

#             email_alert += node_alert_text + '\n'

#         elif k == 'disk_type':    # this is a drive alert
#             email_alert += drive_event_heading
#             drive_alert_text = f"""
#             Node number: {alert_data[item]['node_id']}
#             Drive slot: {alert_data[item]['slot']}
#             Drive status: {alert_data[item]['state']}
#             Slot type: {alert_data[item]['slot_type']}
#             Disk type: {alert_data[item]['disk_type']}
#             Disk model: {alert_data[item]['disk_model']}
#             Disk serial number: {alert_data[item]['disk_serial_number']}
#             Disk capacity: {alert_data[item]['capacity']}
#             """

#             email_alert += drive_alert_text + '\n'

# print(email_alert)

In [59]:
#  _____                 _ _ _   _                 _ _ _
# | ____|_ __ ___   __ _(_) | | | | __ _ _ __   __| | (_)_ __   __ _
# |  _| | '_ ` _ \ / _` | | | |_| |/ _` | '_ \ / _` | | | '_ \ / _` |
# | |___| | | | | | (_| | | |  _  | (_| | | | | (_| | | | | | | (_| |
# |_____|_| |_| |_|\__,_|_|_|_| |_|\__,_|_| |_|\__,_|_|_|_| |_|\__, |
#                                                              |___/

def generate_alert_email(alert_data, rest_client):
    """
    Generate email alert and return as string
    """

    qq_version = get_qq_version(rest_client)
    cluster_name = get_cluster_name(rest_client)
    cluster_uuid = get_cluser_uuid(rest_client)
    cluster_time = get_cluster_time(rest_client)
    
    counter = 0
    for objs in alert_data:
        counter += 1
    alert_header = '=' * 19 + ' CLUSTER EVENT ALERT! ' + '=' * 19
    email_alert = f"""{alert_header}\nUnhealthy object(s) found. See below for\
info and engage Qumulo Support in your preferred fashion.

Cluster name: {cluster_name}
Cluster UUID: {cluster_uuid}
Approx. time: {cluster_time} UTC

{counter} Event(s) found:\n\n"""

    node_event_heading = '=' * 23 + ' NODE OFFLINE ' + '=' * 23
    drive_event_heading = '=' * 21 + ' DRIVE UNHEALTHY ' + '=' * 22

    for item in alert_data:
        for k,v in alert_data[item].items():
            if k == 'node_status':    # this is a node alert
                email_alert += node_event_heading
                node_alert_text = f"""
Node number: {alert_data[item]['id']}
Node status: {alert_data[item]['node_status']}
Serial Number: {alert_data[item]['serial_number']}
Node UUID: {alert_data[item]['uuid']}           
Node Type: {alert_data[item]['model_number']}
Qumulo Core Version: {qq_version}"""

                email_alert += node_alert_text + '\n\n'

            elif k == 'disk_type':    # this is a drive alert
                email_alert += drive_event_heading
                drive_alert_text = f"""
Node number: {alert_data[item]['node_id']}
Drive slot: {alert_data[item]['slot']}
Drive status: {alert_data[item]['state']}
Slot type: {alert_data[item]['slot_type']}
Disk type: {alert_data[item]['disk_type']}
Disk model: {alert_data[item]['disk_model']}
Disk serial number: {alert_data[item]['disk_serial_number']}
Disk capacity: {alert_data[item]['capacity']}"""

                email_alert += drive_alert_text + '\n'
    
    email_alert = email_alert.replace('\n', '<br>')
    
    return email_alert

def get_email_settings(config):
    """
    Pull various email settings from config file.
    """
    sender_addr = config['email_settings']['sender_address']
    server_addr = config['email_settings']['server_address']

    email_recipients = []
    for email_addr in config['email_settings']['mail_to']:
        email_recipients.append(email_addr)

    return sender_addr, server_addr, email_recipients

def send_email(config, email_alert):
    """
    Send an email populated with alert information to all email addresses in
    receipients list specified in config.py.
    """
    sender_addr, server_addr, email_recipients = get_email_settings(config)
    clustername = config['cluster_settings']['cluster_name']
    subject = f'Event alert for cluster: {clustername}'
        
    # Compose the email to be sent based off received data.
    mmsg = MIMEText(email_alert, 'html')
    mmsg['Subject'] = subject
    mmsg['From'] = sender_addr
    mmsg['To'] = ', '.join(email_recipients)

    session = smtplib.SMTP(server_addr)
    session.sendmail(sender_addr, email_recipients, mmsg.as_string())
    session.quit()

In [60]:
#  __  __       _
# |  \/  | __ _(_)_ __ 
# | |\/| |/ _` | | '_ \
# | |  | | (_| | | | | |
# |_|  |_|\__,_|_|_| |_|


def main():  
    config = load_config('config.json')
    API_HOSTNAME = config['cluster_settings']['cluster_address']
    API_USERNAME = config['cluster_settings']['username']
    API_PASSWORD = config['cluster_settings']['password']
    rest_client = cluster_login(API_HOSTNAME, API_USERNAME, API_PASSWORD)
#     status_of_nodes = retrieve_status_of_cluster_nodes(rest_client)
#     status_of_drives = retrieve_status_of_cluster_drives(rest_client)
#     cluster_status = combine_statuses_formatting(status_of_nodes, status_of_drives)
#     previous_existed = check_for_previous_state(cluster_status)

#     if previous_existed:
#         changes = compare_states()
#         if changes:
#             alert_data, healthy = check_for_unhealthy_objects()        
#     else:
#         alert_data, healthy = check_for_unhealthy_objects()
    
#     if not healthy:
#         email_alert = generate_alert_email(alert_data, rest_client)
#         send_email(config, email_alert)
#     else:
#         print('New unhealthy objects were NOT found. Closing script') # XXX: Remove after testing
    
#     return 0

In [61]:
# execute script:
main()

In [62]:
# moar testing....

In [63]:
def cluster_login(api_hostname, api_username, api_password):
    """
    Accept api_hostname, api_username and api_password as parameters. Log into
    cluster via Qumulo Rest API. Return rest_client for all future API calls.
    """
    
    try:
        rest_client = RestClient(api_hostname, 8000)
        rest_client.login(api_username, api_password)
        return rest_client
    except OSError as err1:
        sys.exit(f'{err1}\nExiting...')
    except TimeoutError as err2:
        sys.exit(f'{err2}\nExiting...')
    except RequestError as err3:
        print('Invalid credentials. Please check config file & try again.')
#         sys.exit('Exiting...')

config = load_config('config.json')
API_HOSTNAME = config['cluster_settings']['cluster_address']
API_USERNAME = config['cluster_settings']['username']
API_PASSWORD = config['cluster_settings']['password']
rest_client = cluster_login(API_HOSTNAME, API_USERNAME, API_PASSWORD)

In [64]:
try:
    status_of_nodes = retrieve_status_of_cluster_nodes(rest_client)
except TimeoutError as err2:
    print(err2, '\nExiting...')
#     status_of_drives = retrieve_status_of_cluster_drives(rest_client)
#     cluster_status = combine_statuses_formatting(status_of_nodes, status_of_drives)
#     previous_existed = check_for_previous_state(cluster_status)

In [65]:
status_of_nodes

{'nodes': [{'id': 1,
   'node_status': 'online',
   'node_name': 'CoffeeTime-1',
   'uuid': '10a1c7aa-fb99-48a1-8dc3-b34b96777742',
   'model_number': 'QVIRT',
   'serial_number': 'QVIRT'},
  {'id': 2,
   'node_status': 'offline',
   'node_name': 'CoffeeTime-2',
   'uuid': 'cbdea0e3-1659-48af-b15b-e97dbbeefd04',
   'model_number': 'QVIRT',
   'serial_number': ''},
  {'id': 3,
   'node_status': 'online',
   'node_name': 'CoffeeTime-3',
   'uuid': '30cc2b34-93a0-4639-ae0b-ad57b4e5dc43',
   'model_number': 'QVIRT',
   'serial_number': 'QVIRT'},
  {'id': 4,
   'node_status': 'online',
   'node_name': 'CoffeeTime-4',
   'uuid': '8924daaa-fec3-49b2-9ee5-38352cabb8c3',
   'model_number': 'QVIRT',
   'serial_number': 'QVIRT'}]}

In [76]:
# file1 = 'cluster_state.json'
# file2 = 'cluster_state_previous.json'

In [77]:
# with open(file1) as f1, open(file2) as f2:
#     data1, data2 = json.load(f1), json.load(f2)
# #     changes = data1 != data2

In [78]:
def cluster_state_previous_file_cleanup():
    """
    Delete cluster_state_previous.json if it exists after script run.
    """

    if 'cluster_state_previous.json' in os.listdir():
        os.remove('cluster_state_previous.json')

In [79]:
os.listdir()

['cluster_state_unhealthy_devices_TEST.json',
 'API-functionality-testing.ipynb',
 '.DS_Store',
 'app',
 'config.json',
 'env',
 '.gitignore',
 '.ipynb_checkpoints',
 '.git',
 '.vscode']

In [80]:
cluster_state_previous_file_cleanup()

In [86]:
#Alan's stuff
def get_current_state():
    data = {}

    with open('cluster_state.json') as f:
        data = json.load(f)

    return data


def check_for_unhealthy_objects():
    """
    Scan the cluster_state.json file to determine whether or not there are
    unhealthy objects. If there are unhealthy objects, append the data to
    new dict object called alert_data, which will later be used to populate
    the alert. Also return whether or not cluster is healthy as bool.
    """
    healthy = True

    data = get_current_state()
    nodes = data['nodes']

    alert_data = {}
    counter = 1

    # scan through json for offline nodes
    for node in nodes:
        if 'online' not in node['node_status']:
            print('ALERT!! UNHEALTHY NODE FOUND.')  # XXX: Later remove
            alert_data[f'Event {counter}'] = node
            counter += 1
            healthy = False
    # scan through json for unhealthy drives
    for dictobj in data['drives']:
        for k, v in dictobj.items():
            if k == 'state':
                if v != 'healthy':
                    print('ALERT!! UNHEALTHY DRIVE FOUND.')  # XXX: Later remove
                    alert_data[f'Event {counter}'] = dictobj
                    counter += 1
                    healthy = False

    if healthy:
        print('No unhealthy changes found.')

    return alert_data, healthy

In [87]:
node_data = {
  'nodes': [{'id': 1, 'node_status': 'online', 'node_name': 'CoffeeTime-1', 'uuid': '10a1c7aa-fb99-48a1-8dc3-b34b96777742', 'model_number': 'QVIRT', 'serial_number': 'QVIRT'}, {'id': 2, 'node_status': 'offline', 'node_name': 'CoffeeTime-2', 'uuid': 'cbdea0e3-1659-48af-b15b-e97dbbeefd04', 'model_number': 'QVIRT', 'serial_number': 'QVIRT'}, {'id': 3, 'node_status': 'online', 'node_name': 'CoffeeTime-3', 'uuid': '30cc2b34-93a0-4639-ae0b-ad57b4e5dc43', 'model_number': 'QVIRT', 'serial_number': 'QVIRT'}, {'id': 4, 'node_status': 'online', 'node_name': 'CoffeeTime-4', 'uuid': '8924daaa-fec3-49b2-9ee5-38352cabb8c3', 'model_number': 'QVIRT', 'serial_number': 'QVIRT'}], 

  'drives': [{'id': '1.1', 'node_id': 1, 'slot': 1, 'state': 'healthy', 'slot_type': 'SSD', 'disk_type': 'SSD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}, {'id': '1.2', 'node_id': 1, 'slot': 2, 'state': 'healthy', 'slot_type': 'SSD', 'disk_type': 'SSD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}, {'id': '1.3', 'node_id': 1, 'slot': 3, 'state': 'healthy', 'slot_type': 'HDD', 'disk_type': 'HDD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}, {'id': '1.4', 'node_id': 1, 'slot': 4, 'state': 'healthy', 'slot_type': 'HDD', 'disk_type': 'HDD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}, {'id': '1.5', 'node_id': 1, 'slot': 5, 'state': 'healthy', 'slot_type': 'HDD', 'disk_type': 'HDD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}, {'id': '1.6', 'node_id': 1, 'slot': 6, 'state': 'healthy', 'slot_type': 'HDD', 'disk_type': 'HDD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}, {'id': '2.1', 'node_id': 2, 'slot': 1, 'state': 'healthy', 'slot_type': 'SSD', 'disk_type': 'SSD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}, {'id': '2.2', 'node_id': 2, 'slot': 2, 'state': 'unhealthy', 'slot_type': 'SSD', 'disk_type': 'SSD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}, {'id': '2.3', 'node_id': 2, 'slot': 3, 'state': 'healthy', 'slot_type': 'HDD', 'disk_type': 'HDD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}, {'id': '2.4', 'node_id': 2, 'slot': 4, 'state': 'healthy', 'slot_type': 'HDD', 'disk_type': 'HDD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}, {'id': '2.5', 'node_id': 2, 'slot': 5, 'state': 'healthy', 'slot_type': 'HDD', 'disk_type': 'HDD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}, {'id': '2.6', 'node_id': 2, 'slot': 6, 'state': 'healthy', 'slot_type': 'HDD', 'disk_type': 'HDD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}, {'id': '3.1', 'node_id': 3, 'slot': 1, 'state': 'healthy', 'slot_type': 'SSD', 'disk_type': 'SSD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}, {'id': '3.2', 'node_id': 3, 'slot': 2, 'state': 'healthy', 'slot_type': 'SSD', 'disk_type': 'SSD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}, {'id': '3.3', 'node_id': 3, 'slot': 3, 'state': 'healthy', 'slot_type': 'HDD', 'disk_type': 'HDD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}, {'id': '3.4', 'node_id': 3, 'slot': 4, 'state': 'healthy', 'slot_type': 'HDD', 'disk_type': 'HDD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}, {'id': '3.5', 'node_id': 3, 'slot': 5, 'state': 'healthy', 'slot_type': 'HDD', 'disk_type': 'HDD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}, {'id': '3.6', 'node_id': 3, 'slot': 6, 'state': 'healthy', 'slot_type': 'HDD', 'disk_type': 'HDD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}, {'id': '4.1', 'node_id': 4, 'slot': 1, 'state': 'healthy', 'slot_type': 'SSD', 'disk_type': 'SSD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}, {'id': '4.2', 'node_id': 4, 'slot': 2, 'state': 'healthy', 'slot_type': 'SSD', 'disk_type': 'SSD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}, {'id': '4.3', 'node_id': 4, 'slot': 3, 'state': 'healthy', 'slot_type': 'HDD', 'disk_type': 'HDD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}, {'id': '4.4', 'node_id': 4, 'slot': 4, 'state': 'healthy', 'slot_type': 'HDD', 'disk_type': 'HDD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}, {'id': '4.5', 'node_id': 4, 'slot': 5, 'state': 'healthy', 'slot_type': 'HDD', 'disk_type': 'HDD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}, {'id': '4.6', 'node_id': 4, 'slot': 6, 'state': 'healthy', 'slot_type': 'HDD', 'disk_type': 'HDD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}]
}

In [88]:
drives = node_data['drives']

In [89]:
for drive in drives:
    if drive['state'] == 'healthy':
        print('healthy!')
    else:
        print('unhealthy!')

healthy!
healthy!
healthy!
healthy!
healthy!
healthy!
healthy!
unhealthy!
healthy!
healthy!
healthy!
healthy!
healthy!
healthy!
healthy!
healthy!
healthy!
healthy!
healthy!
healthy!
healthy!
healthy!
healthy!
healthy!


In [90]:
def check_cluster_connectivity_with_socket():
    """
    Use socket to verify communication with cluster IP over port specified
    in config.json.
    """
    try:
        host_ip = '10.120.0.34'
        rest_port = 8000
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result_of_check = sock.connect_ex((host_ip, rest_port)) # XXX: Remove?
        sock.close()
        print(f'result of check: {result_of_check}')
    except ConnectionRefusedError as e:
        sys.exit(
            f'ERROR: {e}\nCheck port connectivity & try again. Exiting...'
        )
    except socket.timeout as e:
        sys.exit(f'ERROR: {e}\nCheck connection & try again. Exiting...')

In [91]:
check_cluster_connectivity_with_socket()

result of check: 0


In [92]:
import socket
s = socket.socket()
address = '10.120.0.30'
port = 8000
try:
    connect_attempt = s.connect_ex((address, port)) 
except Exception as e: 
    print("something's wrong with %s:%d. Exception is %s" % (address, port, e))
finally:
    s.close()
print(connect_attempt)

61


In [93]:
s = socket.socket()
address = '10.120.0.30'
port = 8000

In [94]:
s.connect_ex((address, port)) 

61

In [95]:
# Works: 
def netcat(host, port):
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.connect((host, int(port)))
        s.shutdown(socket.SHUT_WR)
        s.close()
    except ConnectionRefusedError as e:
        print(f'Conneciton refused! Error:\n{e}')

In [96]:
def check_cluster_connectivity_with_socket(config_file):
    """
    Use socket to verify communication with cluster IP over port specified
    in config.json.
    """
    try:
        host_ip = config_file['cluster_settings']['cluster_address']
        rest_port = config_file['cluster_settings']['rest_port']
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.connect((host_ip, int(rest_port)))
        sock.shutdown(socket.SHUT_WR)
        sock.close()
    except ConnectionRefusedError as e:
        sys.exit(
            f'ERROR: {e}\nCheck port connectivity & try again. Exiting...'
        )
    except socket.timeout as e:
        sys.exit(f'ERROR: {e}\nCheck connection & try again. Exiting...')

In [97]:
def check_cluster_connectivity_with_socket():
    """
    Use socket to verify communication with cluster IP over port specified
    in config.json.
    """
    try:
        host_ip = '10.120.0.34'
        rest_port = 8000
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.connect((host_ip, int(rest_port)))
        sock.shutdown(socket.SHUT_WR)
        sock.close()
    except ConnectionRefusedError as e:
        sys.exit(
            f'ERROR: {e}\nCheck port connectivity & try again. Exiting...'
        )
    except socket.timeout as e:
        sys.exit(f'ERROR: {e}\nCheck connection & try again. Exiting...')

In [98]:
check_cluster_connectivity_with_socket()

In [99]:
cluster_slots_status = rest_client.cluster.get_cluster_slots_status()

In [101]:
alert_data = {'Event 1': {'id': 2, 'node_status': 'offline', 'node_name': 'CoffeeTime-2', 'uuid': 'cbdea0e3-1659-48af-b15b-e97dbbeefd04', 'model_number': 'QVIRT', 'serial_number': 'QVIRT'}, 'Event 2': {'id': '2.2', 'node_id': 2, 'slot': 2, 'state': 'unhealthy', 'slot_type': 'SSD', 'disk_type': 'SSD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}}

In [104]:
print(alert_data)

{'Event 1': {'id': 2, 'node_status': 'offline', 'node_name': 'CoffeeTime-2', 'uuid': 'cbdea0e3-1659-48af-b15b-e97dbbeefd04', 'model_number': 'QVIRT', 'serial_number': 'QVIRT'}, 'Event 2': {'id': '2.2', 'node_id': 2, 'slot': 2, 'state': 'unhealthy', 'slot_type': 'SSD', 'disk_type': 'SSD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}}


In [106]:
alert_data = {'Event 1': {'id': 3, 'node_status': 'offline', 'node_name': 'CoffeeTime-3', 'uuid': '30cc2b34-93a0-4639-ae0b-ad57b4e5dc43', 'model_number': 'QVIRT', 'serial_number': 'QVIRT'}, 'Event 2': {'id': '2.2', 'node_id': 2, 'slot': 2, 'state': 'unhealthy', 'slot_type': 'SSD', 'disk_type': 'SSD', 'disk_model': 'Virtual_disk', 'disk_serial_number': '', 'capacity': '10467934208'}}

In [107]:
alert_data

{'Event 1': {'id': 3,
  'node_status': 'offline',
  'node_name': 'CoffeeTime-3',
  'uuid': '30cc2b34-93a0-4639-ae0b-ad57b4e5dc43',
  'model_number': 'QVIRT',
  'serial_number': 'QVIRT'},
 'Event 2': {'id': '2.2',
  'node_id': 2,
  'slot': 2,
  'state': 'unhealthy',
  'slot_type': 'SSD',
  'disk_type': 'SSD',
  'disk_model': 'Virtual_disk',
  'disk_serial_number': '',
  'capacity': '10467934208'}}

In [113]:
for entry in alert_data:
    for k in alert_data[entry].items():
        print(k)
        print(k == 'node_status')

('id', 3)
False
('node_status', 'offline')
False
('node_name', 'CoffeeTime-3')
False
('uuid', '30cc2b34-93a0-4639-ae0b-ad57b4e5dc43')
False
('model_number', 'QVIRT')
False
('serial_number', 'QVIRT')
False
('id', '2.2')
False
('node_id', 2)
False
('slot', 2)
False
('state', 'unhealthy')
False
('slot_type', 'SSD')
False
('disk_type', 'SSD')
False
('disk_model', 'Virtual_disk')
False
('disk_serial_number', '')
False
('capacity', '10467934208')
False


In [160]:
cluster_nodes = [{'id': 1, 'node_status': 'online', 'node_name': 'CoffeeTime-1', 'uuid': '10a1c7aa-fb99-48a1-8dc3-b34b96777742', 'label': '00:50:56:bf:68:82', 'model_number': 'QVIRT', 'serial_number': 'QVIRT', 'mac_address': '00:50:56:bf:68:82'}, {'id': 2, 'node_status': 'offline', 'node_name': 'CoffeeTime-2', 'uuid': 'cbdea0e3-1659-48af-b15b-e97dbbeefd04', 'label': '00:50:56:bf:f1:57', 'model_number': 'QVIRT', 'serial_number': '', 'mac_address': ''}, {'id': 3, 'node_status': 'online', 'node_name': 'CoffeeTime-3', 'uuid': '30cc2b34-93a0-4639-ae0b-ad57b4e5dc43', 'label': '00:50:56:bf:3a:d0', 'model_number': 'QVIRT', 'serial_number': 'QVIRT', 'mac_address': '00:50:56:bf:3a:d0'}, {'id': 4, 'node_status': 'online', 'node_name': 'CoffeeTime-4', 'uuid': '8924daaa-fec3-49b2-9ee5-38352cabb8c3', 'label': '00:50:56:bf:5e:37', 'model_number': 'QVIRT', 'serial_number': 'QVIRT', 'mac_address': '00:50:56:bf:5e:37'}]

In [ ]:
# old

node_relevant_fields = [
    'id',
    'node_status',
    'node_name',
    'uuid',
    'model_number',
    'serial_number',
]
temp_list = []
status_of_nodes = {}

for num in range(len(cluster_nodes)):
    new_dict = {}
    for k,v in cluster_nodes[num].items():
        if k in node_relevant_fields:
            new_dict[k] = v
    temp_list.append(new_dict)
status_of_nodes['nodes'] = temp_list

In [171]:
# new
node_relevant_fields = [
    'id',
    'node_status',
    'node_name',
    'uuid',
    'model_number',
    'serial_number',
]
temp_list = []
status_of_nodes = {}

for event, anus in enumerate(cluster_nodes):
    new_dict = {}
    for k, v in event.items():
        if k in node_relevant_fields:
            new_dict[k] = v
    temp_list.append(new_dict)
status_of_nodes['nodes'] = temp_list

AttributeError: 'int' object has no attribute 'items'

In [164]:

    print(v)

{'id': 1, 'node_status': 'online', 'node_name': 'CoffeeTime-1', 'uuid': '10a1c7aa-fb99-48a1-8dc3-b34b96777742', 'label': '00:50:56:bf:68:82', 'model_number': 'QVIRT', 'serial_number': 'QVIRT', 'mac_address': '00:50:56:bf:68:82'}
{'id': 2, 'node_status': 'offline', 'node_name': 'CoffeeTime-2', 'uuid': 'cbdea0e3-1659-48af-b15b-e97dbbeefd04', 'label': '00:50:56:bf:f1:57', 'model_number': 'QVIRT', 'serial_number': '', 'mac_address': ''}
{'id': 3, 'node_status': 'online', 'node_name': 'CoffeeTime-3', 'uuid': '30cc2b34-93a0-4639-ae0b-ad57b4e5dc43', 'label': '00:50:56:bf:3a:d0', 'model_number': 'QVIRT', 'serial_number': 'QVIRT', 'mac_address': '00:50:56:bf:3a:d0'}
{'id': 4, 'node_status': 'online', 'node_name': 'CoffeeTime-4', 'uuid': '8924daaa-fec3-49b2-9ee5-38352cabb8c3', 'label': '00:50:56:bf:5e:37', 'model_number': 'QVIRT', 'serial_number': 'QVIRT', 'mac_address': '00:50:56:bf:5e:37'}


In [169]:
for  in enumerate(cluster_nodes):
    print(v)

{'id': 4, 'node_status': 'online', 'node_name': 'CoffeeTime-4', 'uuid': '8924daaa-fec3-49b2-9ee5-38352cabb8c3', 'label': '00:50:56:bf:5e:37', 'model_number': 'QVIRT', 'serial_number': 'QVIRT', 'mac_address': '00:50:56:bf:5e:37'}
{'id': 4, 'node_status': 'online', 'node_name': 'CoffeeTime-4', 'uuid': '8924daaa-fec3-49b2-9ee5-38352cabb8c3', 'label': '00:50:56:bf:5e:37', 'model_number': 'QVIRT', 'serial_number': 'QVIRT', 'mac_address': '00:50:56:bf:5e:37'}
{'id': 4, 'node_status': 'online', 'node_name': 'CoffeeTime-4', 'uuid': '8924daaa-fec3-49b2-9ee5-38352cabb8c3', 'label': '00:50:56:bf:5e:37', 'model_number': 'QVIRT', 'serial_number': 'QVIRT', 'mac_address': '00:50:56:bf:5e:37'}
{'id': 4, 'node_status': 'online', 'node_name': 'CoffeeTime-4', 'uuid': '8924daaa-fec3-49b2-9ee5-38352cabb8c3', 'label': '00:50:56:bf:5e:37', 'model_number': 'QVIRT', 'serial_number': 'QVIRT', 'mac_address': '00:50:56:bf:5e:37'}
